In [1]:
# !pip install openai
# !pip install tiktoken

In [ ]:
import json
import tiktoken

with open('train_processed.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

tokenizer = tiktoken.encoding_for_model('gpt-4')

# Calculate the total number of tokens for each entry
def calculate_tokens(entry):
    text = f"Category: {entry['category_name']}\nTitle: {entry['title']}\nDescription: {entry['description']}\nAttributes: {entry['attributes']}\nPrice: {entry['price']}"
    tokens = tokenizer.encode(text)
    return len(tokens)
total_tokens = sum(calculate_tokens(entry) for entry in data)
print(f"Total tokens: {total_tokens}")


Total tokens: 3547350


In [ ]:
import json
import tiktoken
import numpy as np

tokenizer = tiktoken.encoding_for_model('gpt-4')

def calculate_tokens(entry):
    text = (f"Category: {entry['category_name']}\n"
            f"Title: {entry['title']}\n"
            f"Description: {entry['description']}\n"
            f"Attributes: {entry['attributes']}\n"
            f"Price: {entry['price']}")
    tokens = tokenizer.encode(text)
    return len(tokens)

def get_question_count(percentile):
    return int((percentile-0.000001)/10) + 5

def calculate_percentiles(data):
    token_counts = [calculate_tokens(entry) for entry in data]
    sorted_token_counts = np.sort(token_counts)
    return token_counts, sorted_token_counts

def assign_questions_based_on_tokens(entry, sorted_token_counts, total_entries):
    token_count = calculate_tokens(entry)
    percentile = np.searchsorted(sorted_token_counts, token_count) / total_entries * 100
    return get_question_count(percentile)

token_counts, sorted_token_counts = calculate_percentiles(data)
total_entries = len(data)


In [ ]:
def preprocess_description(description):
    return description.replace("[NEWLINE]", "\n").strip()
def create_prompt(entry, questions_num):
    prompt = (
        f"На основе следующего объявления сгенерируйте {questions_num} вопросов и соответствующих ответов."
        f"Придумывая вопрос, представьте, что вы играете роль покупателя, который заинтересован в товаре и "
        f"хочет узнать как можно больше, а также, совершить выгодную сделку."
        f"Придумывайте вопросы, которые могли бы задать реальные покупатели, интересующиеся покупкой по этому объявлению. "
        f"Придумав вопрос, ответьте на него, как если бы вы были экспертом по этому товару. "
        f"Отвечая на вопрос, используйте только ту информацию о товаре, которая доступна в объявлении, "
        f"либо же является общеизвестным фактом. "

        f"**Обратите внимание**:\n"
        f"- Вопросы должны быть разной длины и сложности, имитируя реальные вопросы пользователей.\n"
        f"- Ответы должны быть краткими и точными.\n"
        f"- Формат ответа должен быть строгим: 'Вопрос: {{вопрос}}\n', после вопроса - 'Ответ: {{ответ}}'.\n"
        f"- Если невозможно дать ответ на поставленный вопрос, используя информацию о товаре из объявления, "
        f"либо же используя общеизвестные факты, то ответьте 'Пожалуйста, уточните у продавца'"

        f"Далее идет информация об объявлении:\n"
        f"**Объявление**:\n"
        f"Категория: {entry['category_name']}\n"
        f"Заголовок: {entry['title']}\n"
        f"Описание: {preprocess_description(entry['description'])}\n"
        f"Характеристики: {entry['attributes']}\n"
        f"Цена: {entry['price']} руб.\n"
    )
    return prompt


# with open('processed_data.json', 'r', encoding='utf-8') as file:
#     data = json.load(file)
# prompts = [create_prompt(entry, assign_questions_based_on_tokens(entry, sorted_token_counts, total_entries))\
#            for entry in data]
# print(prompts[0])


In [ ]:
from openai import OpenAI
api_key = "YOUR-API-KEY"
client = OpenAI(api_key=api_key)

In [ ]:
import json
import re
client = OpenAI(api_key=api_key)

def parse_qa_pairs(model_output):
    qa_pattern = re.compile(r"Вопрос:\s*(.*?)\s*Ответ:\s*(.*?)($|\n)", re.DOTALL)
    qa_pairs = [{"question": q.strip(), "answer": a.strip()} for q, a, _ in qa_pattern.findall(model_output)]

    return qa_pairs

def generate_qa(entry, client):
    prompt = create_prompt(entry, assign_questions_based_on_tokens(entry, sorted_token_counts, total_entries))
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "Вы - агент поддержки клиентов для платформы Avito. Avito - это платформа "
         "для размещения объявлений о продаже товаров и услуг. Ваша задача — помогать клиентам, генерируя "
         "релевантные вопросы и предоставляя точные ответы на основе объявления. Вы должны отвечать чётко и "
         "вежливо, имитируя реальные вопросы и ответы от пользователей платформы."

        },
        {
            "role": "user",
            "content": prompt
        }
      ]
    )
    model_output = response.choices[0].message.content

    lines = model_output.split("\n")
    qa_pairs = parse_qa_pairs(model_output)

    result = {
        "category": entry['category_name'],
        "title": entry['title'],
        "description": preprocess_description(entry['description']),
        "attributes": entry['attributes'],
        "price": entry['price'],
        "generated_questions": qa_pairs
    }

    return result, model_output


# res, model_out= generate_qa(data[12], client)

data_qa = []
def save(data_qa):
  with open('data_qa.json', 'w', encoding='utf-8') as outfile:
      json.dump(data_qa, outfile, indent=4, ensure_ascii=False)

for i, entry in enumerate(data):
    generated_entry, model_out = generate_qa(entry, client)
    data_qa.append(generated_entry)
    if (i+1) % 20 == 0 and (i+1)%100 != 0:
      print(i)
    if (i+1)%100 == 0:
      print(f'save, i = {i}')
      save(data_qa)



5619
5639
5659
5679
save, i = 5699
5719
5739
5759
5779
save, i = 5799
5819
5839
5859
5879
save, i = 5899


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Вы - агент поддержки клиентов для платформы Avito. Avito - это платформа "
         "для размещения объявлений о продаже товаров и услуг. Ваша задача — помогать клиентам, генерируя "
         "релевантные вопросы и предоставляя точные ответы на основе объявления. Вы должны отвечать чётко и "
         "вежливо, имитируя реальные вопросы и ответы от пользователей платформы."

        },
        {
            "role": "user",
            "content": prompts[0]
        }
    ]
)

print(completion.choices[0].message.content)

1. **Вопросы по теме объявления, на которые можно дать точный ответ:**

   Вопрос: Какова диагональ экрана этого монитора?
   Ответ: Диагональ экрана составляет 18.5 дюймов (47 см).

   Вопрос: Какое разрешение у монитора Philips 191V2SB/6?
   Ответ: Разрешение монитора составляет 1366x768 пикселей.

   Вопрос: Входит ли кабель VGA в комплектацию?
   Ответ: Да, кабель VGA входит в комплектацию монитора.

   Вопрос: Какой угол обзора у этого монитора?
   Ответ: Угол обзора монитора составляет 170 градусов по вертикали и 176 градусов по горизонтали.

   Вопрос: Где находится магазин, в котором продается этот монитор?
   Ответ: Магазин находится в Нижнем Новгороде, по адресу: ул. Чаадаева, д. 26.

2. **Вопросы по теме объявления, на которые нельзя дать точный ответ:**

   Вопрос: Есть ли у монитора какие-либо внешние повреждения или царапины?
   Ответ: Эту информацию следует уточнить у продавца.

3. **Вопросы, которые не относятся к теме объявления:**

   Вопрос: Можете порекомендовать лу

## Заметки

Оказалось, что модель плохо моделирует вопросы, не относящиеся к теме объявления. Практически все такие вопросы на самом деле относятся к теме объявления, и скорее, должны иметь метку "Эту информацию следует уточнить у продавца".   
Пример:    
'question': 'Вопрос: Есть ли гарантия на товары, которые вы продаете?',    
'answer': 'Ответ: Вопрос не относится к теме объявления.'   
    
Поэтому, не будем генерировать такие вопросы. Можно оставить задачу генерации вопросов, не относящихся к теме объявления, другой модели (классификация с BERT, например)


Также, после некоторых экспериментов, был изменен промпт. Из промпта убраны указания о том, сколько % вопросов должны быть релевантными (по теме и тексту объявления), сколько нерелевантными, сколько релевантными, но не имеющими ответа. Основные причины:    
1. Это вынуждает модель больше фокусироваться на структуре, чем на смысле задаваемых вопросов. Это существенно снижает их качество.  
2. Вопросы с ответом "Пожалуйста, уточните у продавца" с новым промптом генерируются тогда, когда действительно являются уместными.